In [1]:
import pandas as pd
import requests

def lade_wetterdaten(start, ende, breite, länge):
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": breite,
        "longitude": länge,
        "start_date": start,
        "end_date": ende,
        "daily": [
            "weathercode"
        ],
        "timezone": "Europe/Berlin"
    }
    antwort = requests.get(url, params=params)
    daten = antwort.json()
    if "daily" not in daten:
        raise ValueError("Wetterdaten konnten nicht geladen werden.")
    return pd.DataFrame({
        "Datum": daten["daily"]["time"],
        "Wettercode": daten["daily"]["weathercode"]
    })

if __name__ == "__main__":
    pfad_baeckerei = "/workspaces/bakery_sales_prediction/5_Datasets/bakery_training_dataset.csv"
    breite = 54.3233
    länge = 10.1394
    start = "2013-07-01"
    ende = "2019-07-31"

    # 1. Bestehende Daten laden
    df_baeckerei = pd.read_csv(pfad_baeckerei, parse_dates=["Datum"])

    # 2. Wetterdaten laden
    df_wetter = lade_wetterdaten(start, ende, breite, länge)
    df_wetter["Datum"] = pd.to_datetime(df_wetter["Datum"])

    # 3. Merge auf Datum
    df_merged = df_baeckerei.merge(df_wetter, on="Datum", how="left")

    # 4. Optional: Ersetze ggf. vorhandene Spalte "Wettercode_fehlt" durch neue Werte
    if "Wettercode_fehlt" in df_merged.columns:
        df_merged["Wettercode_fehlt"] = df_merged["Wettercode"]

    # 5. Abspeichern
    df_merged.to_csv(pfad_baeckerei, index=False)
    print("CSV mit Wettercode aktualisiert.")

CSV mit Wettercode aktualisiert.
